In [ ]:
import requests
import datetime

mseconds_since_epoch = datetime.datetime.now().timestamp()*1000
ra_src = 208
dec_src = 10
search_rad = 10
print('Searching: (%d, %d) in a %d window since %d time' % (ra_src, dec_src, search_rad, int(mseconds_since_epoch)))

response = requests.get('https://astro-colibri.herokuapp.com/cone_search?cone=[%d,%d,%d]&datemin=0&datemax=%d' % \
                        (ra_src, dec_src, search_rad, int(mseconds_since_epoch)))

data = response.json()

correlations = data['sources']
alerts = data['voevents'] 

historical_alerts = []
for event in alerts:
    print('-----')
    #print (event)
    newtime  =  datetime.datetime.utcfromtimestamp(event['timestamp']/1000.0)
    print(newtime, event['ra'], event['dec'],event['err'])
    fancy_name = event['type']
    historical_alerts.append([newtime, event['ra'], event['dec'],event['err'],fancy_name])
    #print(newtime)

In [ ]:
print(historical_alerts)

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.colors import LinearSegmentedColormap
import numpy as mp

two_weeks_ago = datetime.datetime.now() - datetime.timedelta(days=150)
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

color_dict={}
for evt in historical_alerts:
    if evt[4] in color_dict:
        continue
    else:
        color_dict[evt[4]] = cycle[len(color_dict)+1]

print(color_dict)

ra_now=[]
dec_now=[]
color_now=[]
ra_old=[]
dec_old=[]
color_old=[]
for evt in historical_alerts:
    if evt[0]>two_weeks_ago:
        ra_now.append(evt[1])
        dec_now.append(evt[2])
        color_now.append(color_dict[evt[4]])
    else:
        ra_old.append(evt[1])
        dec_old.append(evt[2])
        color_old.append(color_dict[evt[4]])
    
print(ra,dec)

f, (ax, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [3, 1]})
cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
ax.set_xlim((ra_src - search_rad/2.0*1.20), (ra_src + search_rad/2.0*1.20))
ax.set_ylim((dec_src - search_rad/2.0*1.20), (dec_src + search_rad/2.0*1.20))

ax.scatter(ra_now, dec_now, color=color_now, s=100)
ax.scatter(ra_old, dec_old, color=color_old, s=50, alpha=0.2)

for evt in historical_alerts:
    if evt[0]>two_weeks_ago:
        satur = 1.0
    else:
        satur = 0.2
    if evt[3]=='None':
        err = 0.0
    else:
        err = evt[3]
    circle = plt.Circle((evt[1], evt[2]), err, color=color_dict[evt[4]], fill=False, alpha = satur)
    ax.add_patch(circle)

legend_elements = []
for key in color_dict:
    print(key, '-', color_dict[key])
    legend_elements.append(Line2D([0], [0], marker = 'o', color = color_dict[key], label = key))

ax.set_ylabel('DEC (deg)')
ax.set_xlabel('RA (deg)')


ax2.axis('off')
ax2.legend(handles=legend_elements)

plt.savefig('historical_alert_suggetion.png')

In [ ]:
## alternative to plotting simple error circle
import numpy as np

ic_ra = 208.093
ic_dec = 10.034
ic_90_rad = 1.4699

rev2_ra = 207.42
rev2_dec = 10.43
# file of error contours for IC20220928A
# dec [rad], ra [rad] pairs

with open('IC20220928_90_contour.txt') as f:
    lines = f.readlines()
f.close()

contour_x_vals = []
contour_y_vals = []

for item in lines:
    line = item.split()
    contour_x_vals.append(np.rad2deg(float(line[0])))
    contour_y_vals.append(np.rad2deg(float(line[1])))
                          
f, ax = plt.subplots(1, 1)
ax.set_xlim((ic_ra - ic_90_rad/2.0*4.50), (ic_ra + ic_90_rad/2.0*4.50))
ax.set_ylim((ic_dec - ic_90_rad/2.0*3.50), (ic_dec + ic_90_rad/2.0*3.50))

ax.scatter([ic_ra], [ic_dec], color='b', s=100)
ax.scatter([rev2_ra], [rev2_dec], color='r', s=100)

circle = plt.Circle((ic_ra, ic_dec), ic_90_rad, color='b', fill=False)
ax.add_patch(circle)
# now the offline contour
ax.scatter(contour_x_vals, contour_y_vals, color='r', s=10)
legend_elements = [Line2D([0], [0], color = 'b', label = 'Rev0'),
                  Line2D([0], [0], color = 'r', label = 'Rev1 contour')]
ax.legend(handles=legend_elements)


ax.set_ylabel('DEC (deg)')
ax.set_xlabel('RA (deg)')
ax.set_title('IC220929A')
plt.savefig('IC_contours_suggetion.png')
